# Testing GraphQL Handler with Improved Implementation

This notebook tests the improved GraphQL handler implementation.

In [ ]:
import os
import json
import asyncio
import logging
from pathlib import Path
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Add the ingestion package to the path
import sys
sys.path.append('../src')

from ingestion.handlers.graphql.graphql_handler import GraphQLHandler

## Load Environment Variables

First, let's load our environment variables

In [ ]:
# Load environment variables
env_path = Path(__file__).parents[2] / ".platform_config" / "dev_platform" / ".env.liveheats"
load_dotenv(env_path)

# Enable testing mode
os.environ["TESTING"] = "1"

print("Environment variables loaded")
print(f"Using scope: {os.environ['SCOPE']}")

## Set Up Test Configuration

Now let's set up our test configuration

In [ ]:
# Set up test configuration
task_config = {
    "query": """
        query OrganisationAthletes($id: ID!, $page: Int!, $per: Int!) {
            organisationAthletes(id: $id, page: $page, per: $per) {
                athletes {
                    id
                    name
                    dob
                    nationality
                    properties
                    memberships {
                        id
                    }
                }
                totalCount
            }
        }
    """,
    "variables": {
        "id": "1",
        "page": 1,
        "per": 100
    },
    "sink": {
        "type": "local",
        "key_prefix": "lh_organisations"
    }
}

# Set up mock secrets
mock_secrets = {
    "CLIENT_ID": os.environ["CLIENT_ID"],
    "CLIENT_SECRET": os.environ["CLIENT_SECRET"],
    "GRAPHQL_URL": os.environ["GRAPHQL_URL"],
    "AUTH_URL": os.environ["AUTH_URL"],
    "SCOPE": os.environ["SCOPE"]
}

# Print the secrets (hiding sensitive values)
safe_secrets = mock_secrets.copy()
safe_secrets["CLIENT_ID"] = safe_secrets["CLIENT_ID"][:10] + "..."
safe_secrets["CLIENT_SECRET"] = safe_secrets["CLIENT_SECRET"][:10] + "..."
print("Mock secrets:")
print(json.dumps(safe_secrets, indent=2))

## Initialize Handler and Test Authentication

Let's first test just the authentication part

In [ ]:
# Set up temporary data directory
data_dir = Path("/tmp/graphql_test")
data_dir.mkdir(parents=True, exist_ok=True)
os.environ["DATA_DIR"] = str(data_dir)

# Initialize handler
handler = GraphQLHandler(task_config)

# Test authentication
async def test_auth():
    try:
        result = await handler._get_auth_token(mock_secrets)
        print("Successfully authenticated with API")
        print(f"Result: {json.dumps(result, indent=2)}")
    except Exception as e:
        print(f"Auth failed: {str(e)}")

await test_auth()

## Test Full Query Execution

Now let's test executing a query

In [ ]:
# Test query execution
async def test_query():
    try:
        result = await handler.execute_query(mock_secrets)
        print("Query succeeded!")
        print("Result:")
        print(json.dumps(result, indent=2))
    except Exception as e:
        print(f"Query failed: {str(e)}")

await test_query()

## Test Full Data Processing

Finally, let's test the complete data processing pipeline

In [ ]:
# Test full processing
async def test_process():
    try:
        await handler.process_data()
        print("Processing succeeded!")
        
        # Check output files
        output_files = list(data_dir.glob("**/*.json"))
        print(f"\nOutput files:")
        for f in output_files:
            print(f"- {f}")
            
            # Print first few lines of each file
            with open(f) as fp:
                data = json.load(fp)
                print("\nFirst few lines of data:")
                print(json.dumps(data, indent=2)[:500] + "...")
    except Exception as e:
        print(f"Processing failed: {str(e)}")

await test_process()